# Estimate Binomial draw probabilility

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.4020427180880297

Use a flat priors (the default, omitted) for α

In [5]:
P = TransformedLogDensity(as((α = as𝕀,)), p)
∇P = ADgradient(:ForwardDiff, P);

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0012 s/step ...done
MCMC, adapting ϵ (25 steps)
8.6e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0027 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00014 s/step ...done
MCMC, adapting ϵ (200 steps)
6.9e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
6.5e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
7.6e-6 s/step ...done
MCMC (1000 steps)
6.3e-6 s/step ...done


(DynamicHMC.NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.0769036], -3.70412, 2, DoubledTurn, 0.823751, 3), NUTS_Transition{Array{Float64,1},Float64}([0.543755], -3.46309, 1, DoubledTurn, 0.804769, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.249651], -3.78459, 2, DoubledTurn, 0.937578, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.354985], -2.78341, 1, DoubledTurn, 0.98825, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.354985], -3.66888, 1, AdjacentTurn, 0.731417, 3), NUTS_Transition{Array{Float64,1},Float64}([-0.619475], -2.99895, 1, DoubledTurn, 0.930555, 1), NUTS_Transition{Array{Float64,1},Float64}([-0.619475], -3.87541, 1, DoubledTurn, 0.691604, 1), NUTS_Transition{Array{Float64,1},Float64}([-1.15287], -3.9514, 2, DoubledTurn, 0.877628, 3), NUTS_Transition{Array{Float64,1},Float64}([-1.23178], -4.45128, 2, DoubledTurn, 0.984214, 3), NUTS_Transition{Array{Float64,1},Float64}([-1.21584], -4.51478, 2, DoubledTurn, 1.0, 3)  …  NUT

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

277.48265654030035

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.89, min/25%/median/75%/max: 0.1 0.84 0.96 1.0 1.0
  termination: AdjacentTurn => 22% DoubledTurn => 78%
  depth: 1 => 58% 2 => 42%


check the mean

In [11]:
mean(posterior_α)

0.45663470513970733

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*